In [5]:
# to do: update is_canonical setting in continuous code
# port improved checks for already canonical
# fix special_bigger

from library import *

uobs = Observable('u', 1)
vobs = Observable('v', 1)
pobs = Observable('p', 0)
#observables = [uobs, vobs, pobs]
observables = [uobs]

### define all parameters and "make library"
#terms = generate_terms_to(4, observables=observables, max_observables=4)
terms = generate_terms_to(4, observables=observables, max_observables=4)

terms0 = []
terms1 = []
for term in terms:
    if term.rank==0:
        #if term.complexity <= 4:
            terms0.append(term)
    else:
        #if term.complexity <= 4:
            terms1.append(term)

1


In [6]:
print("***Rank 0 library***")
for term in terms0:
    print(term, term.complexity)
print("\n***Rank 1 library***")
for term in terms1:
    print(term, term.complexity)
print(len(terms0), len(terms1))

***Rank 0 library***
1 1
dj rho[u_j] 3
dt dj rho[u_j] 4
rho[u_j] * rho[u_j] 4
rho[u_j * u_j] 3
dt rho[u_j] * rho[u_j] 5
dt rho[u_j * u_j] 4

***Rank 1 library***
rho[u_i] 2
di dj rho[u_j] 4
dj^2 rho[u_i] 4
dt rho[u_i] 3
dt^2 rho[u_i] 4
di rho[u_j] * rho[u_j] 5
dj rho[u_i] * rho[u_j] 5
dj rho[u_j] * rho[u_i] 5
di rho[u_j * u_j] 4
dj rho[u_i * u_j] 4
rho[u_i] * rho[u_j] * rho[u_j] 6
rho[u_i * u_j] * rho[u_j] 5
rho[u_i * u_j * u_j] 4
7 13


In [14]:
cgp = CoarseGrainedPrimitive([uobs, uobs, vobs])
cgp2 = CoarseGrainedPrimitive([uobs, vobs, vobs])
print(cgp)

rho[u * u * v]


In [15]:
print(cgp.index_str([2, 1, 2]))
print(cgp.index_canon([2, 1, 2]))
print(cgp.index_canon([2, 1, 0]))
print(cgp2.index_canon([2, 1, 2]))
print(cgp2.index_canon([2, 1, 0]))

rho[u_k * u_j * v_k]
[1, 2, 2]
[1, 2, 0]
[2, 1, 2]
[2, 0, 1]


In [23]:
labels = ordered_index_list_to_labels([[1, 3, 5], [6, 4, 6], [1, 4, 3], [2, 5, 2]])
print(labels)

{1: [(0, 0), (2, 0)], 3: [(0, 1), (2, 2)], 5: [(0, 2), (3, 1)], 6: [(1, 0), (1, 2)], 4: [(1, 1), (2, 1)], 2: [(3, 0), (3, 2)]}


In [26]:
labels_to_ordered_index_list(labels, [3]*4)

[[1, 3, 5], [6, 4, 6], [1, 4, 3], [2, 5, 2]]